# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath = 'weather_data.csv'
weather_data = pd.read_csv(filepath)

weather_data

,Unnamed: 0,City,Temperature,%Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,0,mar del plata,11.10,60,100,1.79,-38.0023,-57.5575
1,1,bluff,8.78,89,100,4.79,-46.6000,168.3333
2,2,albany,23.09,57,99,0.32,42.6001,-73.9662
3,3,provideniya,7.38,78,100,2.78,64.3833,-173.3000
4,4,ilhabela,23.12,70,9,1.58,-23.7781,-45.3581
...,...,...,...,...,...,...,...,...
565,565,oxapampa,19.94,69,12,1.62,-10.5775,-75.4017
566,566,batagay,18.51,64,67,0.57,67.6333,134.6333
567,567,gokak,24.41,68,100,1.82,16.1667,74.8333
568,568,zastron,13.25,59,99,2.08,-30.3023,27.0840


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps with API key
gmaps.configure(api_key = g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data[['Latitude', 'Longitude']]
humidity = weather_data['%Humidity']

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_data


,Unnamed: 0,City,Temperature,%Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,0,mar del plata,11.10,60,100,1.79,-38.0023,-57.5575
1,1,bluff,8.78,89,100,4.79,-46.6000,168.3333
2,2,albany,23.09,57,99,0.32,42.6001,-73.9662
3,3,provideniya,7.38,78,100,2.78,64.3833,-173.3000
4,4,ilhabela,23.12,70,9,1.58,-23.7781,-45.3581
...,...,...,...,...,...,...,...,...
565,565,oxapampa,19.94,69,12,1.62,-10.5775,-75.4017
566,566,batagay,18.51,64,67,0.57,67.6333,134.6333
567,567,gokak,24.41,68,100,1.82,16.1667,74.8333
568,568,zastron,13.25,59,99,2.08,-30.3023,27.0840


In [6]:
#finding weather: <50% Hummidity, <10% Cloudiness, temp 4 - 21 C.
ideal_weather = weather_data.loc[weather_data['%Humidity'] < 50]
ideal_weather = ideal_weather.loc[ideal_weather['Cloudiness'] <= 10]
ideal_weather = ideal_weather.loc[(ideal_weather['Temperature'] >= 4) & (ideal_weather['Temperature'] <= 21)]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#creating hotel df
hotel_df = ideal_weather
hotel_df['Hotel Name'] = ''

hotel_df

,Unnamed: 0,City,Temperature,%Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Hotel Name
99,99,gravelbourg,14.15,45,5,4.43,49.8834,-106.5512,
438,438,santa isabel,19.66,41,0,0.60,-23.3156,-46.2214,
477,477,mount isa,14.87,33,0,3.60,-20.7333,139.5000,
502,502,worland,19.75,48,0,1.54,44.0169,-107.9554,
512,512,ribeirao pires,19.31,46,0,0.61,-23.7106,-46.4133,
534,534,serenje,20.09,36,4,3.94,-13.2325,30.2352,


In [8]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'type': 'hotel',
          'keyword': 'hotel',
        'key': g_key,
        'radius': 5000}

In [9]:
hotel_list = []
hotel_lat_list = []
hotel_lng_list = []

for index, row in hotel_df.iterrows():
    
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params['location'] = f'{lat},{lng}'
    
    response = requests.get(base_url, params=params).json()
    
    #print(response)
    
    try:
        hotel = response['results'][1]['name']
        hotel_lat = response['results'][1]['geometry']['location']['lat']
        hotel_lng = response['results'][1]['geometry']['location']['lng']
        hotel_list.append(hotel)
        hotel_lat_list.append(hotel_lat)
        hotel_lng_list.append(hotel_lng)
    
    except:
        print('No hotel found')

In [10]:
hotel_df['Hotel Name'] = hotel_list

hotel_df

,Unnamed: 0,City,Temperature,%Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Hotel Name
99,99,gravelbourg,14.15,45,5,4.43,49.8834,-106.5512,Gravelbourg Inn
438,438,santa isabel,19.66,41,0,0.60,-23.3156,-46.2214,Hotel Estância Santa Isabel
477,477,mount isa,14.87,33,0,3.60,-20.7333,139.5000,Central Point Motel
502,502,worland,19.75,48,0,1.54,44.0169,-107.9554,Days Inn by Wyndham Worland
512,512,ribeirao pires,19.31,46,0,0.61,-23.7106,-46.4133,Hotel Capriccio Mauá
534,534,serenje,20.09,36,4,3.94,-13.2325,30.2352,SERENJE SOCH INN


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

KeyError: 'Country'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
